In [35]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame


In [36]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [37]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model-rf"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model-rf
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [38]:
val testModel = PipelineModel.load("/home/jovyan/models/spark-ml-model-rf")


testModel = pipeline_dddb4568b910


pipeline_dddb4568b910

In [39]:
val predictionsDF = testModel.transform(inputDF)
predictionsDF.printSchema()


root
 |-- tweet: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



predictionsDF = [tweet: string, words: array<string> ... 4 more fields]


[tweet: string, words: array<string> ... 4 more fields]

In [43]:
val getNegative = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

val predictions_neg = testModel
    .transform(inputDF)
    .withColumn("negative_probability", getNegative($"probability"))
    .select( $"negative_probability")



getNegative = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))
predictions_neg = [negative_probability: double]


[negative_probability: double]

In [45]:

val PD = predictions_neg.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show()
}.start()




PD = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@32b498c2


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@32b498c2

+--------------------+
|negative_probability|
+--------------------+
|  0.5079832589933295|
|  0.4998981906985482|
|  0.5144744106785156|
|  0.5144744106785156|
|  0.4377270318681079|
|  0.5267365540365857|
|  0.4799711748655723|
|  0.5285949589710808|
|  0.5580346098130536|
|  0.5144744106785156|
| 0.49485959221153664|
|  0.4383618265031248|
|  0.5032428419783137|
|  0.5079832589933295|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
|  0.5144744106785156|
|  0.5094358121915039|
| 0.49376741841683514|
|  0.5144744106785156|
|   0.534374861215584|
|  0.5144744106785156|
|  0.5471666853609675|
|  0.5144744106785156|
|  0.5421280868739559|
|  0.5144744106785156|
|  0.5094358121915039|
| 0.44369744173805137|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
|  0.5038659681395237|
| 0.49264848515574655|
| 0.48962560051643556|
|  0.5144744106785156|
| 0.49376741841683514|
| 0.49864955534212185|
|  0.4377

+--------------------+
|negative_probability|
+--------------------+
|  0.5144744106785156|
| 0.47572526125001824|
|  0.4958453007529098|
|  0.5042018204964029|
|  0.5421280868739559|
|  0.5144744106785156|
|  0.4315340616549178|
|   0.540800192088722|
|  0.4388141151152002|
|  0.4809666484951598|
| 0.49151769859802663|
|   0.486479100111015|
|  0.4377270318681079|
|  0.5042018204964029|
|  0.5079832589933295|
|  0.5340750219984324|
|  0.5285949589710808|
|  0.5144744106785156|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
| 0.49800885028321273|
| 0.47793914615674316|
| 0.49880601690384674|
| 0.47725911243254504|
|  0.4967925695308413|
|  0.5267365540365857|
|   0.505191963864342|
|  0.4483430980315061|
|  0.5144744106785156|
|  0.5577827515243656|
|  0.5221038072858947|
|  0.5082814404653254|
|  0.5144744106785156|
|  0.5144744106785156|
|  0.5144744106785156|
|  0.5082814404653254|
|  0.4768921230288303|
+--------------------+

+--------

+--------------------+
|negative_probability|
+--------------------+
|  0.5144744106785156|
| 0.48206199658778975|
| 0.48537834747828423|
|  0.5144744106785156|
|  0.4464250943641666|
|  0.4998981906985482|
|  0.4876898128061489|
| 0.46344600013957865|
|  0.5108722385856928|
|  0.4377270318681079|
|  0.5144744106785156|
|  0.5409737151477773|
|  0.5144744106785156|
|  0.5144744106785156|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
| 0.42844458505393435|
| 0.43967823078685625|
|  0.4377270318681079|
|  0.5475923648187224|
|  0.5047916708679137|
|  0.5094358121915039|
|  0.5144744106785156|
|  0.4948742956877036|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
| 0.49880601690384674|
|  0.4823860400233243|
|  0.4967926329151717|
|  0.5094358121915039|
|  0.5183223687889681|
| 0.41864243911107096|
| 0.49485959221153664|
|  0.4998981906985482|
| 0.45801848703923664|
|  0.5285949589710808|
|  0.5144

+--------------------+
|negative_probability|
+--------------------+
|  0.4876898128061489|
|  0.4984223737555327|
|  0.4927284112931605|
|  0.4541457222494187|
|  0.5144744106785156|
|  0.4377270318681079|
|  0.5471666853609675|
|  0.5485003047101921|
|  0.5082814404653254|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
| 0.43268843338109636|
|  0.4377270318681079|
| 0.48006471527863886|
|  0.5079832589933295|
|  0.4690622294128312|
| 0.48537834747828423|
|   0.420983011552051|
| 0.41116625174937854|
| 0.48338327471401643|
|  0.5144744106785156|
|  0.5144744106785156|
|  0.5079832589933295|
|  0.5193125121569071|
| 0.48338327471401643|
|  0.5095809967400653|
|  0.5471666853609675|
|  0.5460128310351576|
|  0.5144744106785156|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
| 0.49899899365115175|
|  0.4948742956877036|
|  0.4377270318681079|
|  0.4998981906985482|
|  0.5577827515243656|
|  0.5150

+--------------------+
|negative_probability|
+--------------------+
|  0.5079832589933295|
|  0.5144744106785156|
| 0.43034817647453905|
| 0.49899899365115175|
|  0.5406755336757814|
|  0.4506689426913287|
|  0.5144744106785156|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
|  0.5029446605063178|
|  0.4679336417871598|
| 0.48336603834157293|
|  0.4934070390133621|
|  0.4259461108425914|
|  0.4927284112931605|
|  0.5577827515243656|
|  0.5144744106785156|
|  0.5144744106785156|
|  0.5017902887801393|
|  0.5447106371378925|
|  0.5471666853609675|
|  0.5458612119373056|
|  0.5144744106785156|
|  0.5144744106785156|
| 0.49007371504233993|
|  0.5144744106785156|
|  0.5232919373844334|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
|  0.4157679873263029|
|  0.5144744106785156|
|  0.5471666853609675|
| 0.48338327471401643|
| 0.49683433628359525|
|  0.5294849075976236|
|  0.5144744106785156|
|  0.4315

+--------------------+
|negative_probability|
+--------------------+
| 0.42097622021994774|
|  0.5077389082122685|
|  0.4471791130889148|
|  0.4540253464352985|
|  0.5266175963906827|
|  0.4872488445949137|
|  0.5144744106785156|
|  0.5042018204964029|
|  0.4377270318681079|
|  0.4890985099051777|
|  0.5144744106785156|
| 0.48537834747828423|
|  0.5314069222628344|
|  0.5032428419783137|
|  0.5250904768419138|
|  0.5250904768419138|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
|  0.4315340616549178|
|  0.5017902887801393|
|  0.5144744106785156|
|  0.5294849075976236|
|  0.5144744106785156|
|  0.5144744106785156|
|  0.5285949589710808|
|  0.4721134498412116|
|  0.5420229884262325|
| 0.49880601690384674|
| 0.48537834747828423|
| 0.41481283644131856|
|  0.5144744106785156|
|  0.5616812248098265|
|  0.5082814404653254|
|  0.5144744106785156|
|  0.5553337604329995|
+--------------------+

+--------------------+
|negative_probability|
+--------

+--------------------+
|negative_probability|
+--------------------+
|  0.5094358121915039|
|  0.4259461108425914|
|  0.5144744106785156|
|  0.5144744106785156|
|   0.502503742188451|
|  0.4377270318681079|
|  0.5033444774565587|
|  0.5749376294744726|
|  0.5144744106785156|
| 0.45239319820092616|
|  0.4294715690084085|
|  0.5049106285138167|
|  0.4925153661367105|
|  0.5042018204964029|
|  0.5253407598381984|
|  0.5347658658496443|
|  0.4377270318681079|
|  0.4633811321263749|
|  0.5144744106785156|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
|   0.394516817876847|
|  0.4434004249901363|
|  0.5144744106785156|
| 0.47466891168608266|
|  0.4236810375980825|
|  0.5079832589933295|
|  0.5193125121569071|
|  0.5082814404653254|
|  0.5376058001231707|
|  0.5250904768419138|
|  0.5144744106785156|
|  0.4377270318681079|
|  0.5267365540365857|
|  0.5205435838233955|
| 0.42844458505393435|
| 0.49030148122998557|
+--------------------+

+--------

+--------------------+
|negative_probability|
+--------------------+
|  0.4927284112931605|
| 0.46864886537874567|
|  0.5094358121915039|
|  0.4377270318681079|
|   0.579753861595696|
|  0.5144744106785156|
|  0.5042018204964029|
|  0.5144744106785156|
|  0.5144744106785156|
|  0.5144744106785156|
|  0.5347658658496443|
|   0.505191963864342|
|  0.5095392470444936|
|  0.5144744106785156|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
|  0.5347658658496443|
|  0.4377270318681079|
|   0.505191963864342|
|  0.5144744106785156|
|  0.4957471081643684|
|  0.5138767600631913|
|  0.4727147502278487|
|  0.5144744106785156|
| 0.43268843338109636|
|  0.5403972954153076|
|  0.5144744106785156|
|  0.4465443184538979|
|  0.5132837703019565|
|  0.5144744106785156|
|  0.5144744106785156|
|  0.5144744106785156|
|   0.502503742188451|
|  0.5252076408191624|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
|  0.4621